In [16]:
import os
import glob
import json
import quickstats
import sys, numpy as np
from quickstats.components import AnalysisBase

In [17]:
# make sure hh combination fw path is defined
hh_comb_fw_path = os.environ.get('hh_combination_fw_path', None)

assert hh_comb_fw_path

gen_command = True

In [72]:
from quickstats.maths.numerics import is_integer, pretty_value
from pprint import pprint
def create_job(string, key, suf=''):
    tmp = string.split(',')
    poi1 = tmp[0].split('=')[0]
    poi2 = tmp[1].split('=')[0]
    poi1_tmp = tmp[0].split('=')[1].split('_')
    poi2_tmp = tmp[1].split('=')[1].split('_')
    poi1_range = [float(p) for p in poi1_tmp[0:2]]
    poi1_step = float(poi1_tmp[2])
    poi2_range = [float(p) for p in poi2_tmp[0:2]]
    poi2_step = float(poi2_tmp[2])

    poi1_nsteps = int((poi1_range[1] - poi1_range[0]) / poi1_step)
    poi2_nsteps = int((poi2_range[1] - poi2_range[0]) / poi2_step)

#     # slice poi1 with full poi2 range
#     if poi1_nsteps < poi2_nsteps and poi2_nsteps > 12:
#         poi1, poi1_range, poi1_step, poi1_nsteps, poi2, poi2_range, poi2_step, poi2_nsteps = poi2, poi2_range, poi2_step, poi2_nsteps, poi1, poi1_range, poi1_step, poi1_nsteps
        
    poi_slice_up = np.arange(poi1_range[0], poi1_range[1], poi1_step).round(decimals=2)
    new_dict = {}
    for i in range(poi1_nsteps):
        key_name = key+f'_job{suf}{i+1}'
        value = f"^{poi1}={poi_slice_up[i]}_{pretty_value(poi_slice_up[i]+poi1_step)}_{poi1_step},{poi2}={pretty_value(poi2_range[0])}_{pretty_value(poi2_range[1])}_{poi2_step}^"
        new_dict[key_name] = value
    return new_dict


In [19]:
def return_combine_command(paths, config):
    options = {
        "input_dir"    : paths['output'],
        "resonant_type": config['resonant_type'],
        "channels"     : ",".join(config['channels']),
        "file_expr"    : f"\"{config['file_expr']}\"",
        "config"       : paths['task_options'],
        "scheme"       : paths['correlation_schemes'],
        "parallel"     : config['parallel'],
        "skip-limit"   : ""
    }
    if config["blind"]:
        options["blind"] = ""
    else:
        options["unblind"] = ""
    if config["experimental"]:
        options["experimental"] = ""
    else:
        options["official"] = ""
    if config['cache']:
        options['cache'] = ""
    else:
        options['no-cache'] = ""
    if 'prefix' in config:
        options['prefix'] = config['prefix']
    if config['type'] == 'xsec':
        options['minimizer_options'] = os.path.join(hh_comb_fw_path, "configs/minimizer_options/fix_xs_uncertainty.json")

    command_str = "HHComb combine_ws " + " ".join([f"--{key} {value}" for key, value in options.items()])
    print(command_str+' &')
    return command_str



# Step 1: SM point

In [20]:
# specify the input timestamp to use
timestamp = "20220520_noSgHparam"
withbr = None

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_SM.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v12.json"),
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1,
    'type'         : 'mu'
}
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : "",
}

## process channels

In [6]:

if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] = ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
    
    
command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str1)

HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_SM.yaml --parallel -1 --skip-limit  --unblind  --experimental  --cache 


## combine ws

In [7]:
command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_SM.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --unblind  --experimental  --cache  &


## obtain path

In [21]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0.root")
    rescaled_ws_paths[channel]  = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbbb/0.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbtautau/0.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbyy/0.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0.root'}

## Step 2: SM ranking

In [115]:
paths['output']

'/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation'

In [117]:
from quickstats.components import ExtendedModel
channel = 'combined'
model = ExtendedModel(rescaled_ws_paths[channel])
param_sets = model.get_constrained_nuisance_parameters()


INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root"
INFO: Loaded workspace "combWS"
INFO: Loaded model config "ModelConfig"
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_3_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_2_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_2_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_3_Xhh_2_model_bbbb
INFO: Activated binned likelihood attribute for ggf_17_dEta_1_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_17_dEta_2_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_17_dEta_3_Xhh_1_model_bbbb
INFO:

In [119]:
condor = True
if condor:
    print('Save jobrank.txt')
    original_stdout = sys.stdout
    sys.stdout = open('jobrank.txt', 'w')
    
for p in param_sets:
    if p.GetName().startswith('gamma_'):
        continue
    channel_command_str = f'quickstats run_pulls -i {rescaled_ws_paths["combined"]} --poi xsec_br -o {paths["output"]}/ranking/{channel} -p {p.GetName()}'

    print("Arguments =", channel_command_str.replace(' ', '____'))
    print("Queue 1")
    print()

if condor: 
    sys.stdout = original_stdout
    

Save jobrank.txt


['ATLAS_EG_RESOLUTION_ALL', 'ATLAS_EG_SCALE_AF2', 'ATLAS_EG_SCALE_ALL_bbtautau', 'ATLAS_EG_SCALE_ALL_bbyy', 'ATLAS_EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR', 'ATLAS_EL_EFF_Iso_TOTAL_1NPCOR_PLUS_UNCOR', 'ATLAS_EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR', 'ATLAS_EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR', 'ATLAS_FF_TTBAR_ANTITAU_DIFF', 'ATLAS_FF_TTBAR_ANTITAU_TAU25EF_EIGEN0', 'ATLAS_FF_TTBAR_ANTITAU_TAU25RNN_EIGEN0', 'ATLAS_FF_TTBAR_ANTITAU_TAU25_EIGEN0', 'ATLAS_FF_TTBAR_EXTRAPOL_FF', 'ATLAS_FF_TTBAR_EXTRAPOL_TF_FT_1P', 'ATLAS_FF_TTBAR_EXTRAPOL_TF_FT_3P', 'ATLAS_FF_TTBAR_EXTRAPOL_TF_FT_PS', 'ATLAS_FF_TTBAR_TAU25EF_EIGEN0', 'ATLAS_FF_TTBAR_TAU25EF_EIGEN1', 'ATLAS_FF_TTBAR_TAU25EF_EIGEN2', 'ATLAS_FF_TTBAR_TAU25RNN_EIGEN0', 'ATLAS_FF_TTBAR_TAU25RNN_EIGEN1', 'ATLAS_FF_TTBAR_TAU25RNN_EIGEN2', 'ATLAS_FF_TTBAR_TAU25_EIGEN0', 'ATLAS_FF_TTBAR_TAU25_EIGEN1', 'ATLAS_FF_TTBAR_TAU25_EIGEN2', 'ATLAS_FF_TTBAR_TAU25_EIGEN3', 'ATLAS_FT_EFF_Eigen_B_0', 'ATLAS_FT_EFF_Eigen_B_1', 'ATLAS_FT_EFF_Eigen_B_2', 'ATLAS_FT_EFF_Eige

## Step 3: SM studies

In [22]:
# specify the input timestamp to use
timestamp = "20220520_noSgHparam"
withbr = None

for corr in ["nonres_kl_v12.json", "nonres_kl_v12_decor.json", "nonres_kl_v12_decor_corrtop2.json", "nonres_kl_v12_decor_corrtop2exp.json"]:
    for job in ['mu', 'xsec']:
        
        paths = {
            'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
            'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022", f"nonres_SM.yaml"),
            'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", corr),
            'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
        }

        config = {
            'resonant_type': 'nonres',
            'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
            'file_expr'    : '<mass[F]>',
            'blind'        : False,
            'cache'        : True,
            'experimental' : True,
            'parallel'     : -1,
            'type'         : job,
            'prefix'       : corr[:-5]+'_'+job+'_',
        }
        options = {
            "input_dir"    : paths['input'],
            "resonant_type": config['resonant_type'],
            "channels"     : ",".join(config['channels']),
            "outdir"       : paths['output'],
            "file_expr"    : f"\"{config['file_expr']}\"",
            "config"       : paths['task_options'],
            "parallel"     : config['parallel'],
            "skip-limit"   : "",
        }

        if config['type'] == 'xsec':
            options['minimizer_options'] = os.path.join(hh_comb_fw_path, "configs/minimizer_options/fix_xs_uncertainty.json")



        if config["blind"]:
            options["blind"] = ""
        else:
            options["unblind"] = ""
        if config["experimental"]:
            options["experimental"] = ""
        else:
            options["official"] : ""
        if config['cache']:
            options['cache'] = ""
        else:
            options['no-cache'] = ""


        command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
#         print('\n## process channels', corr, job)
#         print(command_str1)
        print('\n## combine ws', corr, job)
        command_str = return_combine_command(paths, config)
        
        



## combine ws nonres_kl_v12.json mu
HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_SM.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --unblind  --experimental  --cache  --prefix nonres_kl_v12_mu_ &

## combine ws nonres_kl_v12.json xsec
HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonr

## Step 2: Cross section scans

In [80]:
# specify the input timestamp to use
timestamp = "20220520_with_BR_decorrelation"
# timestamp = "20220520_noSgHparam"

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_kl_kt_xsection.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v12.json"),
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>_kl',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1,
    'type'         : 'xsec'
}
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : "",
}

if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] = ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] = ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str1)

if config['type'] == 'xsec':
    options['fix'] = f"\"THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0\""


HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520_with_BR_decorrelation --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --parallel -1 --skip-limit  --unblind  --experimental  --cache 


In [81]:
command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --unblind  --experimental  --cache  --minimizer_options /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/minimizer_options/fix_xs_uncertainty.json &


In [82]:
os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root")

'/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/combined/nonres/*/0_kl.root'

In [83]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0_kl.root")
    rescaled_ws_paths[channel] = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_kl.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbtautau/0_kl.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbyy/0_kl.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root'}

In [94]:
# use '_job' + suffix to split scan range
param_expr_maps = {
    'klambda_job1': "^klambda=-15_-10_0.2^",
    'klambda_job2': "^klambda=-10_-5_0.2^",
    'klambda_job3': "^klambda=-5_0_0.2^",
    'klambda_job4': "^klambda=0_5_0.2^",
    'klambda_job5': "^klambda=5_10_0.2^",
    'klambda_job6': "^klambda=10_15_0.2^",
    'klambda_job7': "^klambda=15_20_0.2^",
    'klambda_final': "^klambda=-15_20_0.2^",
    'k2v_job1': "^k2V=-2_1_0.2^",
    'k2v_job2': "^k2V=1_4_0.2^",
    'k2v_job3': "^k2V=0.5_1.5_0.05^",
    'k2v_final': "^k2V=-2_4_0.2,k2V=0.5_1.5_0.05^",
    'klambdak2v_final': "^klambda=-15_20_0.2,k2V=-2_4_0.2^",
    'k2vkv_final': "^k2V=-2_6_0.2,kV=-3_3_0.2^",
}

param_expr_maps.update(create_job('klambda=-15_20_0.2,k2V=-2_4_0.2', 'klambdak2v'))
param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=-3_-2_0.2', 'k2vkv', suf='a'))
param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=-2_-1_0.2', 'k2vkv', suf='b'))
param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=-1_0_0.2', 'k2vkv', suf='c'))
param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=0_1_0.2', 'k2vkv', suf='d'))
param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=1_2_0.2', 'k2vkv', suf='e'))
param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=2_3_0.2', 'k2vkv', suf='f'))
pprint(param_expr_maps)

{'k2v_final': '^k2V=-2_4_0.2,k2V=0.5_1.5_0.05^',
 'k2v_job1': '^k2V=-2_1_0.2^',
 'k2v_job2': '^k2V=1_4_0.2^',
 'k2v_job3': '^k2V=0.5_1.5_0.05^',
 'k2vkv_final': '^k2V=-2_6_0.2,kV=-3_3_0.2^',
 'k2vkv_joba1': '^k2V=-2.0_-1.8_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba10': '^k2V=-0.2_0_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba11': '^k2V=-0.0_0.2_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba12': '^k2V=0.2_0.4_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba13': '^k2V=0.4_0.6_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba14': '^k2V=0.6_0.8_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba15': '^k2V=0.8_1_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba16': '^k2V=1.0_1.2_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba17': '^k2V=1.2_1.4_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba18': '^k2V=1.4_1.6_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba19': '^k2V=1.6_1.8_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba2': '^k2V=-1.8_-1.6_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba20': '^k2V=1.8_2_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba21': '^k2V=2.0_2.2_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba22': '^k2V=2.2_2.4_0.2,kV=-3_-2_0.2^',
 'k2vkv_joba23': '^k2V=2.4_2.6_0.2,kV=-3_-2_0.

In [95]:
options_map = [{
    'poi'         : 'xsec_br',
    'data'        : 'combData',
    'unblind'     : '',
    'fix'         : "^THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0,alpha_THEO_XS_PDFalphas_ggf=0,alpha_THEO_XS_PDFalphas_vbf=0,alpha_THEO_XS_SCALEMTop_ggf=0,alpha_THEO_XS_SCALEMTop_vbf=0^",
}, {
    'poi'         : 'mu_HH_VBF',
    'data'        : 'combData',
    'unblind'     : '',
    'fix'         : "^THEO_XS_fixmu_*ggFHH=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_SCALE_HH_VBF*=0,alpha_THEO_XS_PDFalphas_vbf=0,alpha_THEO_XS_SCALEMTop_vbf=0^",
}
]
command_str_map = {}

condor = True
if condor:
    print('Save job2.txt')
    original_stdout = sys.stdout
    sys.stdout = open('job2.txt', 'w')

for channel in rescaled_ws_paths:
    print(f"# Channel: {channel}")
    input_path  = rescaled_ws_paths[channel]
    for key, expr in param_expr_maps.items():
        if key.split('_')[0] == 'k2v':
            options = options_map[1]
        else:
            options = options_map[0]
        
        if condor and 'final' in key:
            continue
        if not condor and 'job' in key:
            continue
        if 'final' in key:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, key.replace('_final', ''))
        else:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, (key+'1')[:key.find('_job')])

        channel_options = {"input_path": input_path, "outdir": outdir, "param_expr": expr, **options}
        channel_command_str = "quickstats limit_scan " + \
                              " ".join([f"--{key} {value}" for key, value in channel_options.items()])
        command_str_map[channel] = channel_command_str


        if not condor:
            print(channel_command_str.replace('^', '"'))
        else:
            print("Arguments =", channel_command_str.replace(' ', '____'))
            print("Queue 1")
    print()

if condor: 
    sys.stdout = original_stdout

Save job2.txt
